In [198]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
import parmap
import numpy as np
import pandas as pd
import os
import shutil
import cv2
import scipy.io as sio
from Specgram import Specgram
import csv

import glob2

from numba import jit
from sklearn.svm import SVC # "Support vector classifier"

from utility_DLC import *
                     

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [199]:
# SELECT SPECIFIC RECORDING
root_dir = '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/'
#root_dir = '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2am_May2_Week4_30Hz/'

# manually add labels from DLC
labels = ['left_paw','right_paw','nose','lever','right_ear','jaw','tongue']


In [8]:
# plot single mouse statistics (Fig 1A/B)
plot_lever_traces_02_04_single_animal(root_dir)


/home/cat/code/widefieldPredict/self_initiated_paper/utility_DLC.py:1227: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "r--" (-> color='r'). The keyword argument will take precedence.
  plt.plot([t[0], t[-1]], [40,40],'r--',c='black')
/home/cat/code/widefieldPredict/self_initiated_paper/utility_DLC.py:1232: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "r--" (-> color='r'). The keyword argument will take precedence.
  plt.plot([0, 0], [0,100],'r--', linewidth=3, c='black')
/home/cat/code/widefieldPredict/self_initiated_paper/utility_DLC.py:1252: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "r--" (-> color='r'). The keyword argument will take precedence.
  plt.plot([t[0], t[-1]], [40,40],'r--',c='black')


In [43]:
#########################################################################
########### Statistics over all recordings in dataset; Fig 1C/D #########
#########################################################################

main_dir = '/media/cat/4TBSSD/yuki/'

animals = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

#
excluded_mice = []
limited_mice = [] # mice that show substantial differnces abruptly at one point in time- prsumably stroke

# 
fig = plt.figure()
Y1fit, Y1data, X = plot_number_04_all_mice_all_sessions(main_dir, '04', animals, fig)

Y2fit, Y2data,X = plot_number_04_all_mice_all_sessions(main_dir, '02', animals, fig)

# # STATISTICS OF PULLS (Test of results 1)
# array =[]
# for k in range(len(X)):
#     print (len(X[k]))
#     array.append(len(X[k]))
# print ("Mean # of days: ", np.mean(array))
# print (" STD # of days: ", np.std(array))


/home/cat/code/widefieldPredict/self_initiated_paper/utility_DLC.py:1135: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  break
/home/cat/code/widefieldPredict/self_initiated_paper/utility_DLC.py:1135: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  break
/home/cat/code/widefieldPredict/self_initiated_paper/utility_DLC.py:1135: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  break
/home/cat/code/widefieldPredict/self_initiated_paper/utility_DLC.py:1135: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is

In [47]:
# PLOT RATIO OF REWARD/UNREWARD (Fig 1E)
colors = [
'black','blue','red','green', 'cyan','orange','brown','slategrey','darkviolet','darkmagenta',
'lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']
for k in range(len(Y1fit)):
    #ax=plt.subplot(3,3,k+1)
    #plt.plot(X[k],Y1[k])
    #plt.plot(X[k],Y2[k])
    x_norm = X[k]/X[k][-1]
    plt.plot(x_norm,
             Y1fit[k]/Y2fit[k],
             linewidth=10, c=colors[k])
    #break
plt.ylim(0,3)
#plt.xlim(0,140)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title("Rewarded/Unrewarded",fontsize=40)
plt.xlabel("Day of Study", fontsize=40)
plt.ylabel("Ratio", fontsize=40)
plt.show()

In [63]:
# Figs 1D,E - distribution of pulls per 
print ("Y1", Y1fit[0], Y1data[0])
means_reward=[]
means_unreward=[]
width = 25
bins =np.arange(0,250,width)
for k in range(len(Y1fit)):
    ax = plt.subplot(2,3,k+1)
    data = Y1data[k]
    y = np.histogram(data, bins = bins)
    plt.bar(y[1][:-1]+width/2.,y[0]/np.max(y[0]),width-1, color= 'blue',
           alpha=.5)
    data = Y2data[k]
    y = np.histogram(data, bins = bins)
    plt.bar(y[1][:-1]+width/2.,y[0]/np.max(y[0]),width-1, color= 'red',
           alpha=.5)
    
    #plt.scatter(Y2data[k]*0+k,Y2data[k], s=100,c=colors[k])
    #plt.scatter(k, np.mean(Y2data[k]),s=600,c=colors[k])
    means_reward.append(np.mean(Y1data[k]))
    means_unreward.append(np.mean(Y2data[k]))
    plt.xlim(0,225)

plt.show()

Y1 [27.23177337 27.10072275 26.96967213 26.83862151 26.70757088 26.1833684
 26.05231778 25.92126715 25.79021653 25.39706467 25.26601405 25.13496342
 25.0039128  24.87286218 24.47971031 24.34865969 23.95550783 23.56235596
 23.1692041  23.03815348 22.90710285 22.77605223 22.38290037 22.25184975
 22.12079912 21.9897485  21.85869788 21.46554601 21.33449539 21.20344477
 21.07239415 20.94134353 20.54819166 20.41714104 20.28609042 20.1550398
 19.49978669 19.36873607 19.23768545 19.10663482 18.71348296 18.58243234
 18.45138171 18.32033109 17.79612861 17.53402736 17.27192612 16.87877425
 16.74772363 16.61667301 16.48562239 16.35457177 15.9614199  15.83036928
 15.69931866 15.56826804 15.43721741 14.91301493 14.65091368 14.51986306
 14.38881244 13.99566058 13.86460995 13.60250871 13.47145809 13.07830622
 12.9472556  12.81620498 12.55410374] [ 2 25 60  0  4 51 11 41  1 16 43 38 12 75  2 84 12 31  2 34 24 36 22  7
 23 25 26 49 26  8  9 16 12  5 11 11 21 17 14 13 11  5 18  9  5 23 22 14
 23 15 42  0

In [64]:
# Figs 1 Supplemnetary 

width = 10
y1 = np.histogram(means_reward, bins=np.arange(0,200,width))
plt.bar(y1[1][:-1], y1[0], width-1,color='blue',alpha=.5,label="Rewarded")
y2 = np.histogram(means_unreward, bins=np.arange(0,200,width))
plt.bar(y2[1][:-1], y2[0],width-1,color='red',alpha=.5,label="Unrewarded")

plt.xlabel("# of pulls",fontsize=40)
plt.xticks(fontsize=40)
plt.legend(fontsize=40)
#plt.ylabel("# of pulls", fontsize=40)
plt.yticks([])
plt.yticks(fontsize=40)
#plt.ylim(-1,200)
plt.xlim(0,200)
plt.show()

# DATA THAT GOES IN TO RESULTS #1
print (np.mean(means_reward))
print (np.std(means_reward))
print (np.mean(means_unreward))
print (np.std(means_unreward))

from scipy import stats

#print (stats.kstest(means_reward, 'norm'))
print (stats.shapiro(means_reward))
print (stats.shapiro(means_unreward))
#print (stats.kstest(means_unreward, 'norm'))


[20.014492753623188, 38.07142857142857, 47.0, 35.869565217391305, 40.357142857142854, 43.85321100917431] [19.10144927536232, 29.547619047619047, 140.73809523809524, 91.45652173913044, 106.83333333333333, 60.403669724770644]
37.52764006812671
8.636162182442508
74.68011472638516
42.83433433925725
ShapiroResult(statistic=0.8755131363868713, pvalue=0.24907895922660828)
ShapiroResult(statistic=0.9570517539978027, pvalue=0.7967551946640015)


In [65]:
# load traces and labels from DLC .csv file
fname = glob2.glob(root_dir + '/*1030000.csv')[0]
traces, labels, traces_nan = load_csv(fname)


data labels:  ['bodyparts', 'left_paw', 'left_paw', 'left_paw', 'right_paw', 'right_paw', 'right_paw', 'nose', 'nose', 'nose', 'lever', 'lever', 'lever', 'right_ear', 'right_ear', 'right_ear', 'jaw', 'jaw', 'jaw', 'tongue', 'tongue', 'tongue']
column vals:  ['coords', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood']
(20028, 22)


In [69]:
# OPTIONAL DLC SCATTER PLOT OVERLAYED ON VIDEO IMAGE VIA IMSHOW(); FIG 1 E/F
root_dir = '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/'
#root_dir = '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2am_May2_Week4_30Hz/'
    
# load traces and labels from DLC .csv file
fname = glob2.glob(root_dir + '/*1030000.csv')[0]
traces, labels, traces_nan = load_csv(fname)

plot_DLC_scatter(traces_nan)


data labels:  ['bodyparts', 'left_paw', 'left_paw', 'left_paw', 'right_paw', 'right_paw', 'right_paw', 'nose', 'nose', 'nose', 'lever', 'lever', 'lever', 'right_ear', 'right_ear', 'right_ear', 'jaw', 'jaw', 'jaw', 'tongue', 'tongue', 'tongue']
column vals:  ['coords', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood']
(20028, 22)
LOADING A HARDFIXED DATASET:  /media/cat/4TBSSD/yuki/IA1/m4v/prestroke/IA1am_May6_Week4_30Hz.m4v


In [60]:
# Supplementary Fig 1 content
# CORRELATE DLC LEVER WITH LEVER ANGLE - USING SUPERRESOLUTION 1MS ALIGNMENT
# compute 1MS CORRELATIONS BETWEEN DLC AND MOTOR LEVER
if False:
    root_dirs = ['/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb17_30Hz/',
                '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/',
                '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2am_May2_Week4_30Hz/']

    shifts = violion_plots_all_sessions(root_dirs)
    print ("Shifts: ", shifts, " sec.")

In [70]:
## plot DLC traces vs. Lever dynamics

# load traces and labels from DLC .csv file
fname = glob2.glob(root_dir + '/*1030000.csv')[0]
traces, labels, traces_nan = load_csv(fname)

#
cor, shift, cor_random, max_cor, abspos_upsampled = correlate_DLC_vs_motor(root_dir, traces,
                                                                plotting=False)
#shift = -shift
print (" SHIFT FROM CORRELATION: ", shift, " sec")

plot_DLC_traces_vs_lever(shift, traces, labels, root_dir)
plt.xlim(100, 200)
plt.show()

print ("TODO: compute / output Correlation values + random shift")

data labels:  ['bodyparts', 'left_paw', 'left_paw', 'left_paw', 'right_paw', 'right_paw', 'right_paw', 'nose', 'nose', 'nose', 'lever', 'lever', 'lever', 'right_ear', 'right_ear', 'right_ear', 'jaw', 'jaw', 'jaw', 'tongue', 'tongue', 'tongue']
column vals:  ['coords', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood']
(20028, 22)
Video frame rate is:  15.0
Max correlation:  [0.5711482]
 SHIFT FROM CORRELATION:  3.414  sec
(20028, 3)
(20028, 3)
(20028, 3)
TODO: compute / output Correlation values + random shift


/home/cat/code/widefieldPredict/self_initiated_paper/utility_DLC.py:58: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "r--" (-> color='r'). The keyword argument will take precedence.
  plt.plot(abstimes_shifted, abspositions, 'r--', linewidth=6,  c='red', label="lever (motor)")


In [72]:
# find the starts of each dataset 
window = 3.0
plotting = False

starts_arrays, traces_filtered = get_starts_arrays(root_dir, 
                                                   window, 
                                                   plotting)

print (labels)
# save triggers:
if True:
    try:
        os.mkdir(root_dir+'/triggers/')
    except:
        pass
    
    np.save(root_dir+"/triggers/starts_arrays.npy", starts_arrays)
    np.save(root_dir+"/triggers/traces_filtered.npy", traces_filtered)
    np.save(root_dir+"/triggers/labels.npy", labels)
    np.save(root_dir+"/triggers/window.npy", window)
    
# print (" # of DLC features tracked: ", len(starts_arrays))
# print ("SHIFT: ", shift)

data labels:  ['bodyparts', 'left_paw', 'left_paw', 'left_paw', 'right_paw', 'right_paw', 'right_paw', 'nose', 'nose', 'nose', 'lever', 'lever', 'lever', 'right_ear', 'right_ear', 'right_ear', 'jaw', 'jaw', 'jaw', 'tongue', 'tongue', 'tongue']
column vals:  ['coords', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood']
(20028, 22)
0 starts local:  85
1 starts local:  85
2 starts local:  85
3 starts local:  91
4 starts local:  86
5 starts local:  67
6 starts local:  85
['left_paw', 'right_paw', 'nose', 'lever', 'right_ear', 'jaw', 'tongue']


/media/cat/4TBSSD/anaconda3/envs/wide/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [73]:
# # find grooming periods
# print ("TODO: Improve heuristics for grooming identificiation")
# fname = glob2.glob(root_dir + '/*1030000.csv')[0]
# traces, labels, traces_nan = load_csv(fname)

# # bsic heuristic for where paws are
# starts_grooming = np.where(traces[0][:,0]>traces[5][:,0])[0]/15.
# print (starts_grooming)

# # compute lockouts 
# starts_local = []
# starts_local.append(starts_grooming[0])
# for k in range(1,starts_grooming.shape[0],1):
#     if starts_grooming[k]-starts_grooming[k-1]>=window:
#         starts_local.append(starts_grooming[k])

# starts_grooming = np.array(starts_local)

# # save triggers:
# if True:
#     try:
#         os.mkdir(root_dir+'/triggers/')
#     except:
#         pass
    
#     np.save(root_dir+"/triggers/starts_grooming.npy", starts_grooming)
    

TODO: Improve heuristics for grooming identificiation
data labels:  ['bodyparts', 'left_paw', 'left_paw', 'left_paw', 'right_paw', 'right_paw', 'right_paw', 'nose', 'nose', 'nose', 'lever', 'lever', 'lever', 'right_ear', 'right_ear', 'right_ear', 'jaw', 'jaw', 'jaw', 'tongue', 'tongue', 'tongue']
column vals:  ['coords', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood']
(20028, 22)
[   2.73333333    3.13333333    3.2           3.26666667    3.4
    3.6           8.            8.06666667    8.13333333    8.2
    9.2           9.4           9.53333333    9.6           9.66666667
    9.73333333    9.8           9.86666667    9.93333333   10.
   10.06666667   10.13333333   10.2          10.26666667   10.33333333
   10.4          10.46666667   10.53333333   10.6          10.66666667
   10.73333333   10.8          10.86666667   16.66666667   16.8
   17.           17.06666667 

In [133]:
# PLOT ALL FEATURE TRIGGERED DATA

# plot lever triggers
#fname = root_dir

# plot DLC triggered DLC traces
window = 10    #sec
plotting = False
if plotting:
    fig=plt.figure()


# load lever traces
abstimes, abspositions, abscodes = load_lever_data(root_dir)
starts_04, starts_04_idx, starts_02, starts_02_idx = find_code04_starts2(abscodes, 
                                                                         abstimes, 
                                                                         abspositions, 
                                                                         lockout=window)
# 
colors = ['green','blue','brown','black','red','cyan','magenta', 'red']  

n_features = 9  
traces_in = traces_filtered.copy()

#
temp_upsampled = abspos_upsampled.copy()

# REPLACE DLC Lever traces with MOTOR lever traces
traces_in = replace_DLC_lever(traces_in, abspos_upsampled, shift)
   
# 
shift_dlc = 0  # Relative shift between all DLC traces is 0
shift_motor = shift.copy() # previously computed shift

features_all = np.arange(7)
feature2 = 0

starts_features = []
# 
for feature1 in features_all:
    
    # find start of each left paw episode
    idxs1 = find_first(starts_arrays[feature1],
                       window)
    
    # 
    feature1_initiations = starts_arrays[feature1][idxs1]

    # plot all data
    if True:
        leads1 = feature1_initiations
        
    # plot only data where feature1 occurs before feature2
    else: 
        print (feature1)
        idxs2 = find_first(starts_arrays[feature2],window)
        feature2_initiations = starts_arrays[feature2][idxs2]

        leads1, leads2, dist1, dist2 = find_leads(feature1_initiations, 
                                                  feature2_initiations, 
                                                  window)


    if leads1.shape[0]<2:
        continue
        
    name_feature = labels[feature1]
    
    starts_features.append(leads1)
    
    if plotting:
        feature_triggered_trace_plots_baseline(9, 
                                           feature1, 
                                           name_feature+ " #"+str(idxs1.shape[0]),
                                           traces_in, 
                                           leads1, 
                                           window, 
                                           shift_dlc, 
                                           colors)

print ("DONE")
if plotting:

#     # PLOT THE CODE TRACES
#     name_feature="code_04"
#     data_out = feature_triggered_trace_plots_baseline(10, 8, name_feature, 
#                                   traces_in, 
#                                   starts_04, 
#                                   window, 
#                                   shift_motor,
#                                   colors)

#     name_feature="code_02"
#     feature_triggered_trace_plots_baseline(10, 9, name_feature, 
#                                   traces_in, 
#                                   starts_02, 
#                                   window, 
#                                   shift_motor,
#                                   colors)
    
#     plt.suptitle(fname + " lockout: "+str(window) + 'sec', fontsize=20)

#     fig=plt.figure()

    traces_preDFF = data_out[1]
    print (len(traces_preDFF))
    print (traces_preDFF.shape)
    ax = plt.subplot(221)
    plt.plot(traces_preDFF[0].T, c='green',alpha=.1)
    ax=plt.subplot(223)
    temp = []
    for k in range(traces_preDFF[0].shape[0]):
        #temp.append(traces_preDFF[0][k]-traces_preDFF[0][k][:window*15].mean(0))
        temp.append(np.abs(traces_preDFF[0][k]-np.mean(traces_preDFF[0][k][:window*15])))
         #       trace_dff = temp3 - np.mean(temp3[:window*15])
    temp = np.array(temp)
    plt.plot(temp.T,c='black',alpha=.1)
    plt.plot(np.median(temp,0),c='black',alpha=1)


    ax = plt.subplot(222)
    plt.plot(traces_preDFF[1].T, c='blue',alpha=.1)

    plt.show()



DONE


In [135]:
print (starts_04)
print (len(starts_features))

[  21.97651005   68.98889399   95.58646798  138.66664886  206.23691297
  218.98544788  252.71151209  289.61603999  306.49061108  319.39793897
  379.23964882  435.32023382  448.96721101  464.03858995  497.77475286
  529.55880094  543.64305997  556.3190999   576.71767402  594.85027885
  631.64020181  645.46790004  664.28000712  675.77195692  704.06145597
  725.0787909   739.34234786  757.59352684  780.77025795  791.34423113
  831.6265831   845.19418716  867.26048994  877.79417109  909.21512294
  925.50640893  945.24303913  966.72725797 1015.43725705 1058.43321896
 1124.36461115 1149.94673586 1172.75199389 1248.66328406]
7


In [196]:
########################################################
########################################################
########################################################

    
# FIND ISOLATED STARTS IN PAIR OF FEATURES : REQUIRED
features_all = [5,3]

# plot paw-leads
feature1 = features_all[0]
feature2 = features_all[1]

# find start of each left paw episode
if True:
    idxs1 = find_first(starts_arrays[feature1],
                       window)
    # 
    feature1_initiations = starts_arrays[feature1][idxs1]

# plot groomings
else:
    idxs1 = find_first(starts_grooming,window)
    print ("total groomings: ", idxs1.shape)
    feature1_initiations = starts_grooming[idxs1]

# find start of each left paw episode
idxs2 = find_first(starts_arrays[feature2],
                   window)
# 
feature2_initiations = starts_arrays[feature2][idxs2]

# plot data where one feature leads the other by at least x seconds;
if False:
    leads1, leads2, dist1, dist2 = find_leads(feature1_initiations,
                                              feature2_initiations, 
                                              window)
    idx = np.where(dist1>=0.0)[0]
    starts_in = leads1[idx]
else:
    starts_in = feature1_initiations
    
# name_feature = labels[feature1]
# print ("surviving Feature1: ", starts_in.shape)

# DATA labels:  ['left_paw', 'right_paw', 'nose', 'lever', 'right_ear', 'jaw', 'tongue']
#                     0            1          2       3        4          5        6

feature=5

included_traces = [0,1,3,5]
trigger_times = starts_04
trigger_times = starts_in
print (shift_motor)
shift = 0
triggered_traces = feature_triggered_trace_plots_baseline_single_panel( 
                              feature,
                              traces_in, 
                              trigger_times, 
                              included_traces,
                              window, 
                              shift, 
                              colors,
                              labels)

3.414


In [206]:
# find grooming periods:
print ("TODO...")
print (" use this function: starts_grooming() ")

TODO...
 use this function: starts_grooming() 


In [207]:
# # plot body part initiated movement vs. lever pull series
# visualize_lever_vs_starts(starts_arrays_lockout, root_dir, shift)


In [202]:
# extract quiet periods
length_quiet_period = 10 # seconds
starts_quiet_periods = find_quiet_periods_specific_length(starts_arrays, 
                                                          length_quiet_period)


Excluding nose movement from assessment of overall body movement
durations
(57,)
# of quiet_periods longer than  10    # (57,)


In [203]:
# FIND DISTRIBUTION OF QUIESCENT PERIODS
window=3
plotting = False
starts_arrays, traces_filtered = get_starts_arrays(root_dir, window, plotting)

_, durations = quiet_periods_histogram(starts_arrays,plotting)

#print ("Durations; ",durations)
fig=plt.figure()
y = np.histogram(durations, bins=np.arange(0,20,1))
plt.bar(y[1][:-1]+0.5,y[0],0.9)
plt.xlim(0,20)
print (np.cumsum(y[0][10:]))
plt.plot(y[1][10:-1],np.cumsum(y[0][10:]),linewidth=5,c='blue')
#plt.ylim(0,20)
plt.xticks(fontsize=40)
plt.ylabel("# of periods", fontsize=40)
plt.xlabel("Duration of quiescent period (sec)",fontsize=40)
plt.yticks(fontsize=40)
plt.semilogy()
plt.show()

data labels:  ['bodyparts', 'left_paw', 'left_paw', 'left_paw', 'right_paw', 'right_paw', 'right_paw', 'nose', 'nose', 'nose', 'lever', 'lever', 'lever', 'right_ear', 'right_ear', 'right_ear', 'jaw', 'jaw', 'jaw', 'tongue', 'tongue', 'tongue']
column vals:  ['coords', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood', 'x', 'y', 'likelihood']
(20028, 22)
0 starts local:  85
1 starts local:  85
2 starts local:  85
3 starts local:  91
4 starts local:  86
5 starts local:  67
6 starts local:  85
Excluding nose movement from assessment of overall body movement
# of quiet_periods longer than  0    # (437,)
[17 28 34 39 44 47 51 51 53]


In [32]:
# plot movement triggered traces;
colors = ['green','blue','brown','black','red','cyan','magenta', 'red']  

# find where 04 - rewarded pulls start;
abstimes, abspositions, abscodes = load_lever_data(root_dir)
starts_04, starts_04_idx, starts_02, starts_02_idx = find_code04_starts2(abscodes, abstimes, abspositions)

# plot DLC triggered DLC traces
window = 6    #sec
n_features=8  
#traces=traces_filtered
traces_in = traces_filtered
#starts_to_process = starts_arrays_lockout
starts_to_process = starts_arrays
fig=plt.figure()
for feature in range(7):
    starts_in = []
    for k in range(1,starts_to_process[feature].shape[0],1):
        if (starts_to_process[feature][k] - starts_to_process[feature][k-1])>window:
            starts_in.append(starts_to_process[feature][k])

    starts_in = np.array(starts_in)

    name_feature = labels[feature]
    feature_triggered_trace_plots(n_features, feature, name_feature,
                                  traces_in, starts_in, window, 0,
                                  colors)
    
# plot lever triggered DLC features
name_feature="code_04"
#shift=3.36
feature_triggered_trace_plots(n_features, 7, 
                              name_feature, traces_in, 
                              starts_04, window, shift,
                              colors)

plt.suptitle(os.path.split(fname)[1], fontsize=20)
plt.show()

print (len(starts_arrays))

7


In [70]:
# plot movement triggered traces;   
colors = ['green','blue','brown','black','red','cyan','magenta', 'red']  

# find where 04 - rewarded pulls start;
abstimes, abspositions, abscodes = load_lever_data(root_dir)
starts_04, starts_04_idx, starts_02, starts_02_idx = find_code04_starts2(abscodes, abstimes, abspositions)
    
window = 3
print ("sfhit: ", shift)
# plot lever triggered DLC features
print ("window: ", window)
included_traces = [0,1,3,6]
fig=plt.figure()
if True:
    feature = 6
    name_feature=labels[feature]

    # compute lockout
#     starts_in = []
#     for k in range(1,starts_arrays[feature].shape[0],1):
#         if (starts_arrays[feature][k] - starts_arrays[feature][k-1])>window:
#                  starts_in.append(starts_arrays[feature][k])
#     starts_in = np.array(starts_in)

    starts_in = starts_arrays_lockout[feature]
    # starts_in = starts_04
    
    shift=0
    # shift=3.36
    triggered_traces = feature_triggered_trace_plots_single_panel( 
                                  name_feature, traces, traces_filtered,
                                  starts_in, window, shift,
                                  colors, included_traces,
                                  labels, feature)
else:
    shift=3.36
    feature=3
    triggered_traces = feature_triggered_trace_plots_single_panel( 
                              name_feature, traces, traces_filtered,
                              starts_04, window, shift,
                              colors, included_traces,
                              labels, feature)


plt.suptitle(os.path.split(fname)[1], fontsize=20)
plt.show()

print (len(starts_arrays))
    
print (len(triggered_traces))

sfhit:  0
window:  3
7
4


In [26]:
fig=plt.figure()
id_ = 0
print (triggered_traces[id_].shape)
plt.plot(triggered_traces[id_].T,c='black',alpha=.1)
mean = np.median(triggered_traces[id_].T,1)
plt.plot(mean,c='red')
plt.show()

(2547, 90)


In [91]:
print (traces[0][:,2])
for k in range(1,traces[0].shape[0],1):
    if traces[0][k][2]<0.9:
        traces[0][k][0]=traces[0][k-1][0]
        traces[0][k][1]=traces[0][k-1][1]
        

[0.00723302 0.00737786 0.01083824 ... 1.         1.         1.        ]


In [99]:
import ruptures as rpt

# generate signal
# n_samples, dim, sigma = 1000, 3, 4
# n_bkps = 4  # number of breakpoints
# signal, bkps = rpt.pw_constant(n_samples, dim, n_bkps, noise_std=sigma)
# print (signal.shape)
# print (bkps)
bkps=[0]
signal = np.sqrt(traces[0][:,0]**2+traces[0][:,1]**2)[:1000]

# detection
algo = rpt.Pelt(model="rbf").fit(signal)
result = algo.predict(pen=10)

# display
rpt.display(signal, bkps, result)
plt.show()


In [97]:
rpt.display(signal, bkps, result)
plt.show()

In [76]:
data = np.load('/home/cat/Downloads/chan_0_data.npy')
print (data.shape)
plt.plot(data)
plt.show()

(6000000,)


In [291]:
# cov_auto_samp(X,delta)/cov_auto_samp(X,0) = auto correlation
def cov_auto_samp(X,delta):
    N = len(X)
    Xs = np.average(X)
    autoCov = 0.0
    times = 0.0
    for i in np.arange(0, N-delta):
        autoCov += (X[i+delta]-Xs)*(X[i]-Xs)
        times +=1
    return autoCov/times

n_vals = 100
cov_matrix = np.zeros((n_vals,n_vals),'float32')
data = np.load('/media/cat/4TBSSD/data/synthetic/ari/data_raw_30000hz_300sec_20hz_to_50hz.npy')
print (data.shape)
data = data[10000:11000]

for k in range(n_vals):
    print (k)
    for p in range(n_vals):
        #cov_matrix[k,p] = cov_auto_samp(data[p:],k)
        temp = np.correlate(data[k:],data[p:],mode='valid')[0]
        cov_matrix[k,p] = temp

plt.imshow(cov_matrix)
plt.show()

(9000000,)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [292]:
#temp_cov = np.cov(data, data)
temp_cov= np.matmul(data[None].T, data[None])
sig = np.sqrt(np.diag(temp_cov))
temp_cov = temp_cov/(sig[:,None]*sig[None])

/home/cat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [293]:
plt.imshow(temp_cov)
plt.show()

In [20]:
data = np.load('/home/cat/Downloads/2018-11-12-1_vision_spike_train(2).npy',allow_pickle=True)
print (data)

{4096: array([    1993,     5799,     9058, ..., 35991975, 35994233, 35997354],
      dtype=int32), 7681: array([   11982,    12702,    12788, ..., 35990456, 35993746, 35994371],
      dtype=int32), 3586: array([     173,     9899,    12027, ..., 35949748, 35958876, 35964147],
      dtype=int32), 7171: array([     860,     4074,     9124, ..., 35955009, 35959649, 35999936],
      dtype=int32), 5251: array([    5264,    10431,    20529, ..., 35971961, 35981300, 35992286],
      dtype=int32), 3077: array([    2175,     2601,     2722, ..., 35974916, 35978112, 35996765],
      dtype=int32), 2461: array([     996,     1400,     2462, ..., 35996498, 35997626, 35999723],
      dtype=int32), 2057: array([    2738,     4761,     6151, ..., 35904604, 35984068, 35994158],
      dtype=int32), 2387: array([   30715,    36493,    80837, ..., 35999506, 35999699, 35999879],
      dtype=int32), 2405: array([     844,     1566,     2276, ..., 35997093, 35998242, 35999467],
      dtype=int32), 1036: arr

In [19]:
data= np.load('/home/cat/Downloads/2018-12-10-4_vision_spike_train(2).npy',allow_pickle=True)
print (data.shape)
print (data)

UnpicklingError: invalid load key, '\x00'.

In [260]:
def h5_to_npy(h5_fname):

    fname_out = h5_fname.replace('h5','npy')
    if os.path.exists(fname_out)==False:
        f = h5py.File(h5_fname, "r")
        # List all groups
        #print("Keys: %s" % f.keys())

        data = f['df_with_missing']['table']

        traces = []
        for k in range(len(data)):
            traces.append(data[k][1].reshape(-1,3))

        traces=np.array(traces).transpose(1,0,2)
        np.save(fname_out, traces)
     
import glob
import h5py
root_dir = '/media/cat/4TBSSD/yuki_lever-ariadna-2020-07-21/IA1/videos_to_label/prestroke/'
fnames = glob.glob(root_dir+"*.h5")    

from tqdm import trange, tqdm
for fname in tqdm(fnames):
    h5_to_npy(fname)
        
        
        
        

Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewHDF5 ['df_with_missing']>
Keys: <KeysViewH

In [261]:
#######################################################################
####### VISUALIZE DISTRIBUTIONS OF X-coord prediction ##################
#######################################################################

# 
def compute_xloc_distributions(prob_cutoffs, 
                                fname):
    
    data = np.load(fname)
    print (data.shape)
     

    width = 50
    bins = np.arange(0,1280,width)

    ctr=0
    for pcutoff in prob_cutoffs:
        for k in range(data.shape[0]):
            temp = data[k]
            probs= data[k,:,2]
            idx = np.where(probs>pcutoff)[0]
            locs = temp[idx]

            # 
            ax=plt.subplot(3,7,ctr+1)
            if ctr%7==0:
                plt.ylabel(str(pcutoff))

            y = np.histogram(locs[:,0], bins=bins)
    
            plt.bar(y[1][:-1], y[0], 
                    width-1,
                   color='black',
                   alpha=.5)
    
            plt.semilogy()
            plt.ylim(1,21000)
            plt.title(names[k]+" "+str(idx.shape[0]),fontsize=12)
            ctr+=1
    plt.suptitle(os.path.split(fname)[1])
    plt.show()

#     
clrs = ['blue','red','yellow','green', 'magenta','pink','cyan']
#names =  ['left_paw', 'right_paw', 'nose', 'lever', 'right_ear', 'jaw', 'tongue']
names = ['left_paw', 'right_paw', 'nose','jaw','right_ear','tongue','lever']

# 
fname = '/media/cat/4TBSSD/yuki_lever-ariadna-2020-07-21/IA1/videos_to_label/prestroke/IA1pm_Feb5_30HzDLC_resnet50_yuki_leverJul21shuffle1_38000.npy'
prob_cutoffs = [0.01,0.1,0.9]

#
compute_xloc_distributions(prob_cutoffs, fname)

(7, 19907, 3)


<ipython-input-261-198dcd46c14c>:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.semilogy()


In [264]:
####################################################################
################## MAKE LABELED VIDEOS #############################
####################################################################

clrs = ['blue','red','yellow','green', 'magenta','pink','cyan']


# 
traces = np.load('/media/cat/4TBSSD/yuki_lever-ariadna-2020-07-21/IA1/videos_to_label/prestroke/IA1pm_Feb5_30HzDLC_resnet50_yuki_leverJul21shuffle1_38000.npy')
print ("Traces: ", traces.shape)

# video sizes
#size_vid = np.array([1280,1024])
size_vid = np.array([1280,720])
size_vid = np.array([640,360])
dot_size = 16
    
# 
fname_in = '/media/cat/4TBSSD/yuki_lever-ariadna-2020-07-21/IA1/videos_to_label/prestroke/IA1pm_Feb5_30Hz.mp4'
original_vid = cv2.VideoCapture(fname_in)

#
start = 15000
end = start+4500

#
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)


# 
fname_out = fname_in.replace('.mp4','_labeled_'+str(start)+"_"+str(end)+'.mp4')
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

# 
dot_size = 5

# setup cutoff 
pcutoff = 0.5

# go through first videos
from tqdm import trange



for n in trange(start, end, 1):
    ret, frame = original_vid.read()

    for f in range(traces.shape[0]):
        if traces[f,n,2]<pcutoff:
            continue
            
        # 
        x = int(traces[f,n,0])
        y = int(traces[f,n,1])
        frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
            matplotlib.colors.to_rgb(clrs[f]))*255.).astype('uint8')
            #matplotlib.colors.to_rgb('white'))*255.).astype('uint8')

    video_out.write(frame)

video_out.release()
original_vid.release()
cv2.destroyAllWindows()
    

  0%|          | 0/4500 [00:00<?, ?it/s]

Traces:  (7, 19907, 3)


100%|██████████| 4500/4500 [00:16<00:00, 274.39it/s]
